In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
from statsmodels.tsa.regime_switching.markov_regression import MarkovRegression
from fredapi import Fred
import sqlite3
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib import gridspec
import matplotlib.dates as mdates
from pandas.tseries.offsets import Day
from decimal import Decimal
import requests
import seaborn as sns
color_pal = sns.color_palette()
from pandas.tseries.offsets import BDay
from multiprocessing import Pool
from tqdm import tqdm

# Parameters
ticker = '^GSPC'
start_date = '1950-01-01'
end_date = '2024-11-15'
split_date = '2024-11-15'

# Download data
data = yf.download(ticker, start=start_date, end=end_date)

# Use adjusted close price
data = data[['Adj Close']]
data.rename(columns={'Adj Close': 'Adj_Close'}, inplace=True)

# Calculate daily returns
data['Index_Returns'] = data['Adj_Close'].pct_change()
data.dropna(inplace=True)

# Separate train and test sets for 'Index_Returns'
train = data.loc[data.index < split_date, 'Index_Returns']
test = data.loc[data.index >= split_date, 'Index_Returns']

# Fit the Markov Switching Model on the Training Data
train_model = MarkovRegression(train, k_regimes=2, trend='c', switching_variance=True)
train_result = train_model.fit()

[*********************100%***********************]  1 of 1 completed


C:\Users\NicholasRatti\AppData\Local\Temp\ipykernel_10036\1988895487.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.rename(columns={'Adj Close': 'Adj_Close'}, inplace=True)
C:\Users\NicholasRatti\AppData\Local\Temp\ipykernel_10036\1988895487.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Index_Returns'] = data['Adj_Close'].pct_change()
C:\Users\NicholasRatti\AppData\Local\Temp\ipykernel_10036\1988895487.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: ht

In [38]:
train_regimes = train_result.smoothed_marginal_probabilities.idxmax(axis=1)
data.loc[train.index, 'Training Regime'] = train_regimes

C:\Users\NicholasRatti\AppData\Local\Temp\ipykernel_10036\1490037833.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[train.index, 'Training Regime'] = train_regimes


In [39]:
data

,Adj_Close,Index_Returns,Training Regime
Date,,,
1950-01-04,16.850000,0.011405,0.0
1950-01-05,16.930000,0.004748,0.0
1950-01-06,16.980000,0.002953,0.0
1950-01-09,17.080000,0.005889,0.0
1950-01-10,17.030001,-0.002927,0.0
...,...,...,...
2024-11-07,5973.100098,0.007431,NaN
2024-11-08,5995.540039,0.003757,NaN
2024-11-11,6001.350098,0.000969,NaN


In [40]:
# Ensure columns are initialized before the loop
data['Recursive_Predictions'] = np.nan
data['Recursive_Predicted_Regime_Label'] = np.nan
data['T2_Recursive_Predictions'] = np.nan  # Initialize for t+2 predictions
data['T2_Recursive_Predicted_Regime_Label'] = np.nan  # Initialize labels for t+2 predictions

# Loop over the test dates, including the last date, using tqdm for progress bar
test_dates = test.index
for i, date in enumerate(tqdm(test_dates)):
    # Expand the training data to include up to the current date
    recursive_train = data.loc[data.index <= date, 'Index_Returns']

    # Fit the Markov Switching Model on the expanded training data
    recursive_model = MarkovRegression(recursive_train, k_regimes=2, trend='c', switching_variance=True)
    try:
        recursive_result = recursive_model.fit(disp=False)
    except Exception as e:
        print(f"Model failed to converge at date {date}: {e}")
        continue  # Skip this date if the model fails to fit

    # Extract smoothed probabilities and last known state probabilities
    smoothed_probs = recursive_result.smoothed_marginal_probabilities
    last_probs = smoothed_probs.iloc[-1].values

    # Extract transition probabilities from the model parameters
    params = recursive_result.params
    p_00 = params['p[0->0]']
    p_10 = params['p[1->0]']
    p_01 = 1 - p_00
    p_11 = 1 - p_10

    # Construct the transition matrix
    transition_matrix = np.array([
        [p_00, p_01],
        [p_10, p_11]
    ])

    # Update state probabilities to predict the next day's regime (t+1)
    state_probs_t1 = np.dot(last_probs, transition_matrix)

    # Determine the most likely regime at t+1
    regime_labels = smoothed_probs.columns.tolist()  # Should be [0, 1]
    most_likely_regime_t1 = regime_labels[np.argmax(state_probs_t1)]

    # Get the next date for prediction (t+1)
    if i + 1 < len(test.index):
        next_date_t1 = test.index[i + 1]
    else:
        # Predicting beyond the available data; estimate next date
        next_date_t1 = date + pd.Timedelta(days=1)
        # Add next_date_t1 to the DataFrame if it doesn't exist
        if next_date_t1 not in data.index:
            data.loc[next_date_t1] = np.nan  # Initialize with NaNs

    # Store the predicted regime in 'Recursive_Predictions' at next_date_t1
    data.at[next_date_t1, 'Recursive_Predictions'] = most_likely_regime_t1

    # Now predict the regime at t+2
    # Update state probabilities to predict the day after next's regime (t+2)
    state_probs_t2 = np.dot(state_probs_t1, transition_matrix)

    # Determine the most likely regime at t+2
    most_likely_regime_t2 = regime_labels[np.argmax(state_probs_t2)]

    # Get the date for prediction at t+2
    if i + 2 < len(test.index):
        next_date_t2 = test.index[i + 2]
    else:
        # Predicting beyond the available data; estimate next date
        next_date_t2 = date + pd.Timedelta(days=2)
        # Add next_date_t2 to the DataFrame if it doesn't exist
        if next_date_t2 not in data.index:
            data.loc[next_date_t2] = np.nan  # Initialize with NaNs

    # Store the predicted regime in 'T2_Recursive_Predictions' at next_date_t2
    data.at[next_date_t2, 'T2_Recursive_Predictions'] = most_likely_regime_t2

    # Determine which regime corresponds to low and high volatility
    variances = [params['sigma2[0]'], params['sigma2[1]']]
    sigma2_0 = variances[0]
    sigma2_1 = variances[1]

    if sigma2_0 < sigma2_1:
        regime_mapping = {0: 'Low Volatility', 1: 'High Volatility'}
    else:
        regime_mapping = {0: 'High Volatility', 1: 'Low Volatility'}

    # Map the predicted regime to labels for t+1
    predicted_label_t1 = regime_mapping[most_likely_regime_t1]
    data.at[next_date_t1, 'Recursive_Predicted_Regime_Label'] = predicted_label_t1

    # Map the predicted regime to labels for t+2
    predicted_label_t2 = regime_mapping[most_likely_regime_t2]
    data.at[next_date_t2, 'T2_Recursive_Predicted_Regime_Label'] = predicted_label_t2

# Display the recursive predictions
print(data.loc[test.index, ['Index_Returns', 'Recursive_Predictions', 'Recursive_Predicted_Regime_Label', 'T2_Recursive_Predictions', 'T2_Recursive_Predicted_Regime_Label']].head())

C:\Users\NicholasRatti\AppData\Local\Temp\ipykernel_10036\4221826898.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Recursive_Predictions'] = np.nan
C:\Users\NicholasRatti\AppData\Local\Temp\ipykernel_10036\4221826898.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Recursive_Predicted_Regime_Label'] = np.nan
C:\Users\NicholasRatti\AppData\Local\Temp\ipykernel_10036\4221826898.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc

            Index_Returns  Recursive_Predictions  \
Date                                               
2015-01-02      -0.000340                    NaN   
2015-01-05      -0.018278                    0.0   
2015-01-06      -0.008893                    0.0   
2015-01-07       0.011630                    0.0   
2015-01-08       0.017888                    0.0   

           Recursive_Predicted_Regime_Label  T2_Recursive_Predictions  \
Date                                                                    
2015-01-02                              NaN                       NaN   
2015-01-05                   Low Volatility                       NaN   
2015-01-06                   Low Volatility                       0.0   
2015-01-07                   Low Volatility                       0.0   
2015-01-08                   Low Volatility                       0.0   

           T2_Recursive_Predicted_Regime_Label  
Date                                            
2015-01-02               

In [41]:
data

,Adj_Close,Index_Returns,Training Regime,Recursive_Predictions,Recursive_Predicted_Regime_Label,T2_Recursive_Predictions,T2_Recursive_Predicted_Regime_Label
Date,,,,,,,
1950-01-04,16.850000,0.011405,0.0,NaN,NaN,NaN,NaN
1950-01-05,16.930000,0.004748,0.0,NaN,NaN,NaN,NaN
1950-01-06,16.980000,0.002953,0.0,NaN,NaN,NaN,NaN
1950-01-09,17.080000,0.005889,0.0,NaN,NaN,NaN,NaN
1950-01-10,17.030001,-0.002927,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
2024-11-11,6001.350098,0.000969,NaN,1.0,High Volatility,1.0,High Volatility
2024-11-12,5983.990234,-0.002893,NaN,0.0,Low Volatility,1.0,High Volatility
2024-11-13,5985.379883,0.000232,NaN,0.0,Low Volatility,0.0,Low Volatility


In [42]:
# Check for NaNs in 'Index_Returns' and drop them
returns = data['Index_Returns'].dropna()

# Fit Markov Switching Model
model = MarkovRegression(returns, k_regimes=2, trend='c', switching_variance=True)
result = model.fit()
print(result.summary())

# Add regime to the data
data.loc[returns.index, 'Full_Vol_Regime'] = result.smoothed_marginal_probabilities.idxmax(axis=1)

C:\Users\NicholasRatti\anaconda3\envs\msm_env\lib\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: A date index has been provided, but it has no associated frequency information and so will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)


                        Markov Switching Model Results                        
Dep. Variable:          Index_Returns   No. Observations:                18838
Model:               MarkovRegression   Log Likelihood               63437.163
Date:                Thu, 14 Nov 2024   AIC                        -126862.326
Time:                        16:01:39   BIC                        -126815.264
Sample:                             0   HQIC                       -126846.884
                              - 18838                                         
Covariance Type:               approx                                         
                             Regime 0 parameters                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0007   5.65e-05     11.530      0.000       0.001       0.001
sigma2      3.997e-05   8.07e-07     49.547      0.0

C:\Users\NicholasRatti\AppData\Local\Temp\ipykernel_10036\2357773592.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.loc[returns.index, 'Full_Vol_Regime'] = result.smoothed_marginal_probabilities.idxmax(axis=1)


In [43]:
data

,Adj_Close,Index_Returns,Training Regime,Recursive_Predictions,Recursive_Predicted_Regime_Label,T2_Recursive_Predictions,T2_Recursive_Predicted_Regime_Label,Full_Vol_Regime
Date,,,,,,,,
1950-01-04,16.850000,0.011405,0.0,NaN,NaN,NaN,NaN,0.0
1950-01-05,16.930000,0.004748,0.0,NaN,NaN,NaN,NaN,0.0
1950-01-06,16.980000,0.002953,0.0,NaN,NaN,NaN,NaN,0.0
1950-01-09,17.080000,0.005889,0.0,NaN,NaN,NaN,NaN,0.0
1950-01-10,17.030001,-0.002927,0.0,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...
2024-11-11,6001.350098,0.000969,NaN,1.0,High Volatility,1.0,High Volatility,0.0
2024-11-12,5983.990234,-0.002893,NaN,0.0,Low Volatility,1.0,High Volatility,0.0
2024-11-13,5985.379883,0.000232,NaN,0.0,Low Volatility,0.0,Low Volatility,0.0


In [44]:
# Non-Recursive Non-Updating Parameter Set

# Extract the smoothed probabilities for the training data
smoothed_probs = train_result.smoothed_marginal_probabilities

# Extract the last known state probabilities (at the end of the training data)
last_probs = smoothed_probs.iloc[-1].values  # Convert to NumPy array

# Regime labels (from smoothed probabilities)
regime_labels = smoothed_probs.columns.tolist()  # Should be [0, 1]

# Extract transition probabilities from the model parameters
params = train_result.params

# Transition probabilities
p_00 = params['p[0->0]']  # Probability of staying in Regime 0
p_10 = params['p[1->0]']  # Probability of moving from Regime 1 to Regime 0

# Probabilities of transitioning to Regime 1
p_01 = 1 - p_00           # Probability of moving from Regime 0 to Regime 1
p_11 = 1 - p_10           # Probability of staying in Regime 1

# Construct the transition matrix
transition_matrix = np.array([
    [p_00, p_01],  # Transitions from Regime 0
    [p_10, p_11]   # Transitions from Regime 1
])

# Regime-specific variances
variances = [params['sigma2[0]'], params['sigma2[1]']]

# Initialize columns for state probabilities and predicted regime in the data DataFrame
data['State_Prob_Regime0'] = np.nan
data['State_Prob_Regime1'] = np.nan
data['Predicted_Regime'] = np.nan

# Initialize state probabilities with the last known probabilities from the training data
state_probs = last_probs.copy()

# Loop over each date in the test set within the data DataFrame
for date in test.index:
    # Update state probabilities using the transition matrix
    state_probs = np.dot(state_probs, transition_matrix)
    
    # Store the state probabilities in the DataFrame
    data.at[date, 'State_Prob_Regime0'] = state_probs[0]
    data.at[date, 'State_Prob_Regime1'] = state_probs[1]
    
    # Determine the most likely regime
    most_likely_regime = regime_labels[np.argmax(state_probs)]
    
    # Store the predicted regime in the DataFrame
    data.at[date, 'Predicted_Regime'] = most_likely_regime

# Determine which regime corresponds to low volatility and high volatility
sigma2_0 = variances[0]
sigma2_1 = variances[1]

if sigma2_0 < sigma2_1:
    regime_mapping = {0: 'Low Volatility', 1: 'High Volatility'}
else:
    regime_mapping = {0: 'High Volatility', 1: 'Low Volatility'}

# Map the predicted regimes to labels
data['Predicted_Regime_Label'] = data['Predicted_Regime'].map(regime_mapping)

# # Display the first few rows of the data DataFrame for the test period
# print(data.loc[test.index, ['Index_Returns', 'State_Prob_Regime0', 'State_Prob_Regime1', 'Predicted_Regime', 'Predicted_Regime_Label']].head())
# Create the 'Blended_Test_Training_Regime' by merging 'Training Regime' and 'Predicted_Regime' based on date condition
data['Blended_Test_Training_Regime'] = None  # Initialize the column with None

# Set values from 'Training Regime' for dates before split_date
data.loc[data.index < split_date, 'Blended_Test_Training_Regime'] = data['Training Regime']

# Set values from 'Predicted_Regime' for dates on or after split_date
data.loc[data.index >= split_date, 'Blended_Test_Training_Regime'] = data['Predicted_Regime']

# Display the DataFrame with the new 'Blended_Test_Training_Regime' column
print(data[['Training Regime', 'Predicted_Regime', 'Blended_Test_Training_Regime']])

            Training Regime  Predicted_Regime Blended_Test_Training_Regime
Date                                                                      
1950-01-04              0.0               NaN                          0.0
1950-01-05              0.0               NaN                          0.0
1950-01-06              0.0               NaN                          0.0
1950-01-09              0.0               NaN                          0.0
1950-01-10              0.0               NaN                          0.0
...                     ...               ...                          ...
2024-11-11              NaN               0.0                          0.0
2024-11-12              NaN               0.0                          0.0
2024-11-13              NaN               0.0                          0.0
2024-11-14              NaN               NaN                          NaN
2024-11-15              NaN               NaN                          NaN

[18840 rows x 3 columns]

C:\Users\NicholasRatti\AppData\Local\Temp\ipykernel_10036\3817632276.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['State_Prob_Regime0'] = np.nan
C:\Users\NicholasRatti\AppData\Local\Temp\ipykernel_10036\3817632276.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['State_Prob_Regime1'] = np.nan
C:\Users\NicholasRatti\AppData\Local\Temp\ipykernel_10036\3817632276.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,c

In [45]:
data

,Adj_Close,Index_Returns,Training Regime,Recursive_Predictions,Recursive_Predicted_Regime_Label,T2_Recursive_Predictions,T2_Recursive_Predicted_Regime_Label,Full_Vol_Regime,State_Prob_Regime0,State_Prob_Regime1,Predicted_Regime,Predicted_Regime_Label,Blended_Test_Training_Regime
Date,,,,,,,,,,,,,
1950-01-04,16.850000,0.011405,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0
1950-01-05,16.930000,0.004748,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0
1950-01-06,16.980000,0.002953,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0
1950-01-09,17.080000,0.005889,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0
1950-01-10,17.030001,-0.002927,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-11,6001.350098,0.000969,NaN,1.0,High Volatility,1.0,High Volatility,0.0,0.763607,0.236393,0.0,Low Volatility,0.0
2024-11-12,5983.990234,-0.002893,NaN,0.0,Low Volatility,1.0,High Volatility,0.0,0.763607,0.236393,0.0,Low Volatility,0.0
2024-11-13,5985.379883,0.000232,NaN,0.0,Low Volatility,0.0,Low Volatility,0.0,0.763607,0.236393,0.0,Low Volatility,0.0


In [46]:
# Flatten MultiIndex columns if necessary
if isinstance(data.columns, pd.MultiIndex):
    data.columns = ['_'.join(filter(None, col)) for col in data.columns]

# Specify the database filename
db_filename = 'v3output/fill_forward_comparison.db'

# Connect to the SQLite database
conn = sqlite3.connect(db_filename)

# Table name for main data export
table_name = 'regime_data'

# Write the flattened DataFrame to the SQLite database
data.to_sql(table_name, conn, if_exists='replace', index=True)

# Verify the table structure after export
cursor = conn.cursor()
cursor.execute(f"PRAGMA table_info({table_name})")
schema = cursor.fetchall()
print("Table schema:", schema)

# Close the connection
conn.close()

Table schema: [(0, 'Date', 'TIMESTAMP', 0, None, 0), (1, 'Adj_Close', 'REAL', 0, None, 0), (2, 'Index_Returns', 'REAL', 0, None, 0), (3, 'Training Regime', 'REAL', 0, None, 0), (4, 'Recursive_Predictions', 'REAL', 0, None, 0), (5, 'Recursive_Predicted_Regime_Label', 'TEXT', 0, None, 0), (6, 'T2_Recursive_Predictions', 'REAL', 0, None, 0), (7, 'T2_Recursive_Predicted_Regime_Label', 'TEXT', 0, None, 0), (8, 'Full_Vol_Regime', 'REAL', 0, None, 0), (9, 'State_Prob_Regime0', 'REAL', 0, None, 0), (10, 'State_Prob_Regime1', 'REAL', 0, None, 0), (11, 'Predicted_Regime', 'REAL', 0, None, 0), (12, 'Predicted_Regime_Label', 'TEXT', 0, None, 0), (13, 'Blended_Test_Training_Regime', 'REAL', 0, None, 0)]


In [47]:
# Filter for rows where 'Recursive_Predictions' and 'Full_Vol_Regime' do not match, excluding NaNs
filtered_data = data[(data['Recursive_Predictions'] != data['Full_Vol_Regime'])].dropna(subset=['Recursive_Predictions', 'Full_Vol_Regime'])

# Flatten MultiIndex columns if necessary
if isinstance(filtered_data.columns, pd.MultiIndex):
    filtered_data.columns = ['_'.join(filter(None, col)) for col in filtered_data.columns]

# Connect to the SQLite database
conn = sqlite3.connect(db_filename)

# Table name for filtered data
new_table_name = 'mismatched_regimes'

# Write the filtered DataFrame to the SQLite database
filtered_data.to_sql(new_table_name, conn, if_exists='replace', index=True)

# Verify the table structure for the filtered data
cursor = conn.cursor()
cursor.execute(f"PRAGMA table_info({new_table_name})")
filtered_schema = cursor.fetchall()
print("Filtered Table schema:", filtered_schema)

# Close the connection
conn.close()

Filtered Table schema: [(0, 'Date', 'TIMESTAMP', 0, None, 0), (1, 'Adj_Close', 'REAL', 0, None, 0), (2, 'Index_Returns', 'REAL', 0, None, 0), (3, 'Training Regime', 'REAL', 0, None, 0), (4, 'Recursive_Predictions', 'REAL', 0, None, 0), (5, 'Recursive_Predicted_Regime_Label', 'TEXT', 0, None, 0), (6, 'T2_Recursive_Predictions', 'REAL', 0, None, 0), (7, 'T2_Recursive_Predicted_Regime_Label', 'TEXT', 0, None, 0), (8, 'Full_Vol_Regime', 'REAL', 0, None, 0), (9, 'State_Prob_Regime0', 'REAL', 0, None, 0), (10, 'State_Prob_Regime1', 'REAL', 0, None, 0), (11, 'Predicted_Regime', 'REAL', 0, None, 0), (12, 'Predicted_Regime_Label', 'TEXT', 0, None, 0), (13, 'Blended_Test_Training_Regime', 'REAL', 0, None, 0)]


In [49]:
# Connect to the SQLite database
conn = sqlite3.connect(db_filename)

# Retrieve the main data table
query = "SELECT * FROM regime_data"
df_main = pd.read_sql_query(query, conn)

# Export the main data to CSV
output_csv_path_main = 'v3output/regime_data_export.csv'
df_main.to_csv(output_csv_path_main, index=False)
print(f"Main data exported to {output_csv_path_main}")

# Retrieve the filtered data table
query_filtered = "SELECT * FROM mismatched_regimes"
df_filtered = pd.read_sql_query(query_filtered, conn)

# Export the filtered data to CSV
output_csv_path_filtered = 'v3output/mismatched_regimes_export.csv'
df_filtered.to_csv(output_csv_path_filtered, index=False)
print(f"Filtered data exported to {output_csv_path_filtered}")

# Close the connection
conn.close()

Main data exported to v3output/regime_data_export.csv
Filtered data exported to v3output/mismatched_regimes_export.csv


In [ ]:
# Function to send Telegram message
def telegram_messenger():
    # Telegram Bot API token and Channel ID
    bot_token = '7328648943:AAH3gHyGf2xgjxBfzPd05F_7IagASgs-Dj0'
    channel_id = '-1002309744206'

    # Connect to the database
    conn = sqlite3.connect('v3output/fill_forward_comparison.db')

    # Query the last two rows from 'Market_Regime' and 'Date' columns
    query = """
    SELECT Date, Market_Regime, Portfolio_Exposure
    FROM financial_data
    ORDER BY DATE(Date) DESC
    LIMIT 2
    """

    # Execute query and load into a DataFrame
    data = pd.read_sql_query(query, conn)

    # Close the database connection
    conn.close()

    # Convert 'Date' to datetime
    data['Date'] = pd.to_datetime(data['Date'], format='%Y-%m-%d')

    # Sort data by Date in ascending order
    data = data.sort_values('Date').reset_index(drop=True)

    # Debugging: Print the data used for the Telegram message
    print("Data used for Telegram message:")
    print(data)

    # Initialize the message variable each time the code runs with bold header
    message = "<b>Your Daily Portfolio Exposure Update</b>\n\n"  # Reset message here
    labels = ["Today's Market Regime", "Tomorrow's Market Regime"]

    # Loop through the DataFrame and format the message
    for index, row in data.iterrows():
        # Format Date
        formatted_date = row['Date'].strftime('%m/%d/%Y').lstrip("0").replace("/0", "/")

        # Add the labeled message for each row with line breaks for better formatting
        message += f"<u>{labels[index]}</u>\n"
        message += f"<i>Date</i>: {formatted_date}\n"
        message += f"<i>Market Regime</i>: {row['Market_Regime']}\n"
        message += f"<i>Portfolio Exposure</i>: {row['Portfolio_Exposure'] * 100:.0f}%\n"

    # Telegram API URL
    api_url = f'https://api.telegram.org/bot{bot_token}/sendMessage'

    # Payload to send with HTML formatting enabled
    payload = {
        'chat_id': channel_id,
        'text': message,
        'parse_mode': 'HTML'  # Enables HTML for bold formatting
    }

    # Send the request
    response = requests.post(api_url, json=payload)

    # Check the response
    if response.status_code == 200:
        print('Message sent successfully!')
    else:
        print(f'Failed to send message. Error: {response.text}')